In [2]:
import pandas as pd
import music21 as m21
import midi

In [49]:
def load_midigram_series(filename):
    midigram = pd.read_csv(filename, delimiter=' ', \
                      names=['time_start', 'time_end', 'track', 'channel', 'pitch', 'velocity'])
    
    starts = pd.DataFrame(dtype=int)
    starts['time'] = midigram['time_start']
    starts['event'] = 'NoteOn'
    starts['velocity'] = midigram['velocity']
    starts['pitch'] = midigram['pitch']
    
    ends = pd.DataFrame(dtype=int)
    ends['time'] = midigram['time_end']
    ends['event'] = 'NoteOff'
    ends['velocity'] = midigram['velocity']
    ends['pitch'] = midigram['pitch']
    
    events = pd.concat([starts, ends])
    events = events.dropna()
    events = events.sort_values(by='time')
    events = events.reset_index()
    
    events['pitch'] = events['pitch'].astype(int)
    events['tick'] = events['time'].diff().fillna(0).astype(int)    
    events['velocity'] = events['velocity'].astype(int)
    
    return events

In [50]:
midigram_series = load_midigram_series('../output/midigrams/aa08839aa14842315923c369fe53f200872d02da.midigram')

In [79]:
def create_midi(series, filename):
    pattern = midi.Pattern()
    track = midi.Track()
    pattern.append(track)
    
    for index, event_data in series.iterrows():
        if event_data['event'] == 'NoteOn':
            event = midi.events.NoteOnEvent(tick=event_data['tick'],\
                                            velocity=event_data['velocity'],\
                                            pitch=event_data['pitch'])
            track.append(event)
        elif event_data['event'] == 'NoteOff':
            event = midi.events.NoteOffEvent(tick=event_data['tick'],\
                                             pitch=event_data['pitch'])
            track.append(event)
            
    track.append(midi.events.EndOfTrackEvent())
    midi.write_midifile(filename, pattern)

In [80]:
create_midi(midigram_series, 'sample.mid')

In [81]:
metadata = pd.read_csv('../output/metadata.csv', index_col=0)
metadata['filename_midigram'] = metadata['filename_mid'].apply(lambda x: x.replace('mid', 'midigram'))

In [82]:
import os
OUTPUT_DIR = '../output/mid_merged'

def create_single_track_midi(filename_midigram):
    midigram_series = load_midigram_series(filename_midigram)
    
    output_filename = os.path.basename(filename_midigram)
    output_filename = os.path.splitext(output_filename)[0] + '_merged.mid'
    output_filename = os.path.join(OUTPUT_DIR, output_filename)
    create_midi(midigram_series, output_filename)
    print('File written: ' + output_filename)

In [83]:
metadata['filename_midigram'].apply(create_single_track_midi)

File written: ../output/mid_merged/6e355a07d5531f372a2f7fb9ac084a120c34440a_merged.mid
File written: ../output/mid_merged/cf0e37cd297b4b030a506f1e1e2c637352dcc12d_merged.mid
File written: ../output/mid_merged/9235e4d8f77349089d45f475fb852ab71c954de8_merged.mid
File written: ../output/mid_merged/6ece453ca4108e48ea6e6d0fec6996441adc09b5_merged.mid
File written: ../output/mid_merged/96b2c5072caf99d739a42bad844e11b6003445aa_merged.mid
File written: ../output/mid_merged/bde0c398ce976bdc4f8dabdb593ebefd695418f8_merged.mid
File written: ../output/mid_merged/86957eab3117ac72af21b65702fadd3fac7668e2_merged.mid
File written: ../output/mid_merged/c7431346d0708b2ee02c2830dc6ecfacb873be6c_merged.mid
File written: ../output/mid_merged/95dc5788932d8b4dab93922879a228543069e2b6_merged.mid
File written: ../output/mid_merged/7195df96d3fa2bcdb441a018188ff6916992e51e_merged.mid
File written: ../output/mid_merged/c6541f31073a5c212aa377571a922ac49ad1f1ec_merged.mid
File written: ../output/mid_merged/d648c971

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2157    None
2158    None
2159    None
2160    None
2161    None
2162    None
2163    None
2164    None
2165    None
2166    None
2167    None
2168    None
2169    None
2170    None
2171    None
2172    None
2173    None
2174    None
2175    None
2176    None
2177    None
2178    None
2179    None
2180    None
2181    None
2182    None
2183    None
2184    None
2185    None
2186    None
Name: filename_midigram, dtype: object